# Preparing for a sample Tableau dash

Using the Tableau-provided FIFA18.csv dataset -- containing player demographics and attribute ratings for 17k+ players in the 2018 version of the game -- to make some reasonably-interesting visualizations:

1) Aging "curves" by position -- selecting the top X% of players in each age-position cohort and plotting them as a scatter, with color by position. Unfortunately, this dataset uses integer-age instead of DOB (so will collapse players from 32 and 0 days to 32 and 364 days into one category). We should be able to see growth/peak/decline, and how attackers peak younger than defenders peak younger than keepers.

2) "Value" by age and position -- could do 95th percentile or something to reduce impact of outliers, then plot as line chart. Want to see if defenders are relatively under-valued (at least in the close-to-real economics of the video game simulation).

3) Overall rating by wages -- not sure if game gets real wage data or guesstimates. Everyone likes to laugh at overpaid professional athletes; underpaid ones are more likely to move (or at least renegotiate). We have nationality and club team, but not the nation/league the club plays in, so it'll be extra work to compare leagues.

We'll start by exploring the data to find out which of these features are already present and which we'll have to engineer.

In [1]:
import pandas as pd
import numpy as np
# import seaborn as sns
from time import sleep
import matplotlib.pyplot as plt
%matplotlib inline
pd.options.display.max_rows = 20
pd.options.display.max_columns = 24

In [2]:
# grab the copy of the data we downloaded
df = pd.read_csv('fifa18_clean.csv')

# let's pretty-print this
df.head()

,Wage (€),Value (€),Name,Age,Photo,Nationality,Flag,Overall,Potential,Club,Club Logo,Special,...,Preferred Positions,RAM,RB,RCB,RCM,RDM,RF,RM,RS,RW,RWB,ST
0,565000,95500000,Cristiano Ronaldo,32,https://cdn.sofifa.org/48/18/players/20801.png,Portugal,https://cdn.sofifa.org/flags/38.png,94,94,Real Madrid CF,https://cdn.sofifa.org/24/18/teams/243.png,2228,...,ST LW,89.0,61.0,53.0,82.0,62.0,91.0,89.0,92.0,91.0,66.0,92.0
1,565000,105000000,L. Messi,30,https://cdn.sofifa.org/48/18/players/158023.png,Argentina,https://cdn.sofifa.org/flags/52.png,93,93,FC Barcelona,https://cdn.sofifa.org/24/18/teams/241.png,2154,...,RW,92.0,57.0,45.0,84.0,59.0,92.0,90.0,88.0,91.0,62.0,88.0
2,280000,123000000,Neymar,25,https://cdn.sofifa.org/48/18/players/190871.png,Brazil,https://cdn.sofifa.org/flags/54.png,92,94,Paris Saint-Germain,https://cdn.sofifa.org/24/18/teams/73.png,2100,...,LW,88.0,59.0,46.0,79.0,59.0,88.0,87.0,84.0,89.0,64.0,84.0
3,510000,97000000,L. Suárez,30,https://cdn.sofifa.org/48/18/players/176580.png,Uruguay,https://cdn.sofifa.org/flags/60.png,92,92,FC Barcelona,https://cdn.sofifa.org/24/18/teams/241.png,2291,...,ST,87.0,64.0,58.0,80.0,65.0,88.0,85.0,88.0,87.0,68.0,88.0
4,230000,61000000,M. Neuer,31,https://cdn.sofifa.org/48/18/players/167495.png,Germany,https://cdn.sofifa.org/flags/21.png,92,92,FC Bayern Munich,https://cdn.sofifa.org/24/18/teams/21.png,1493,...,GK,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [3]:
print(df['Preferred Positions'].nunique())
print(df.info())

802
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17981 entries, 0 to 17980
Data columns (total 74 columns):
Wage (€)               17981 non-null int64
Value (€)              17981 non-null int64
Name                   17981 non-null object
Age                    17981 non-null int64
Photo                  17981 non-null object
Nationality            17981 non-null object
Flag                   17981 non-null object
Overall                17981 non-null int64
Potential              17981 non-null int64
Club                   17733 non-null object
Club Logo              17981 non-null object
Special                17981 non-null int64
Acceleration           17897 non-null float64
Aggression             17913 non-null float64
Agility                17910 non-null float64
Balance                17924 non-null float64
Ball control           17840 non-null float64
Composure              17887 non-null float64
Crossing               17885 non-null float64
Curve                  17908 no

In [4]:
df['Preferred Positions'].where(df['Preferred Positions'].str.contains('ST')).value_counts()

ST              1847
RM ST            111
LM ST            107
CAM ST            88
RW ST             75
CF ST             73
LW ST             67
ST RM             60
ST LM             56
ST CAM            53
                ... 
CF RM ST           1
CM ST LM RM        1
RM CAM ST LM       1
LB ST LM           1
RB RW ST           1
ST CAM RW LW       1
CDM ST CAM         1
CAM ST RM LM       1
LWB LM ST          1
RW LW ST CF        1
Name: Preferred Positions, Length: 208, dtype: int64

### What can we see right away?
OK, so from the header, we can see that it's sorted by Overall, attributes are floats, Goalkeepers don't get rated in some categories. No idea what Special is. 

Preferred Positions is a string but takes 802(!) different values, so we'll need to slim it down to get a useful categorical. Indeed, although a soccer team can only field 1 goalkeeper and 10 outfield players, the combination of positioning on the field and tactical role means that FIFA offers more than twice as many positional designations (which we'll collapse back down). 

We should also note that because there are multiple permutations of the same subset of positions in the Preferred Positions string (e.g., both 'ST RM' and 'RM ST'), we can assume the list is ordered, and that the first position listed is the player's best / most preferred.


# Engineering a player-position feature

There are two methods we could use to assign each player to a single position: 1) use the first one listed in the Preferred Positions field 2) take the highest score from the individual at-position fields. Let's compare them.

In [5]:
case1 = df['Preferred Positions'].map(lambda p: p.split()[0])
print(case1.value_counts())
print(case1.shape)

position_remap = {'CF':'ST'}

case1b = case1.replace(position_remap).rename('primary_pos')
print(case1b.head())

CB     2705
ST     2277
GK     2029
CM     1977
CDM    1396
RM     1350
LM     1326
LB     1309
RB     1195
CAM    1125
RW      429
LW      427
CF      205
LWB     120
RWB     111
Name: Preferred Positions, dtype: int64
(17981,)
0    ST
1    RW
2    LW
3    ST
4    GK
Name: primary_pos, dtype: object


### OK, 
so the good news is that a lot of the "case 1" positions are common/universal, and the list is shortened to 16; since Center Forward is effectively equivalent to Striker, we can reduce it to 15.

Also note that GK appears in this list, but for the other case GKs are NaN for outfield positions (whereas outfield players have GK-skill attribute values).

For now, we'll stick with this 15-position list -- we will reduce it further by collapsing the side-specific positions into one (RW and LW as 'Winger') when looking at the aging curves, but there may be a disparity in valuation (because there are more naturally right-footed players, who are better suited to playing on the right).

In [6]:
# for "case 2", we'll need to subset the dataframe for just the positional-skill columns
# note that we can reorder the list, since our argmax function will tiebreak by first occurrence
# I'll do so to privilege 'central' positions and 4-4-2 labels
positions = ['ST','CM','CB','CDM','CAM','CF','LM','LB','LW','LWB','LAM',\
             'LF','LS','LCB','LCM','LDM','RM','RB','RW','RWB','RAM','RF',\
             'RS','RCB','RCM','RDM']
case2 = df[positions].idxmax(axis=1,skipna=True)
print(case2.value_counts())
print(case2.shape)

CB     4444
ST     2462
LM     1630
CAM    1585
CM     1366
LB     1349
LW     1211
CDM     846
LWB     627
CF      432
dtype: int64
(17981,)


First note that all GKs, because they are null in all columns, are in the CB category -- we could create a new column to identify them (by having those nulls), but we won't need to because...

Surprisingly, all of the right-sided positions disappeared from this list! (And before you ask, I did try running the positions list on a single line to make sure it wasn't because of a typo.) From this, we can conclude that FIFA rates wide players as equally competent on either side of the field, which is certainly unrealistic, although as we saw in case 1 above, the game does list those players' preferred sides.

In addition, there are clear disagreements between the two lists -- it looks like Left-Backs roughly equals the number of players whose first-listed preference above is LB (so the RBs must be allocated elsewhere), whereas the Left Wing-Backs is in excess of LWB + RWB.

### So,
since the results of this method are less interpretable, and we lose the sidedness of wide players (which we may want to use later), we'll move forward using **Case 1**.

## Building the analytic file for Visualization 1

In [7]:
df1 = df.join(case1b,on=None)

vars1 = ['Name','ID','Age','Nationality','Overall','Potential','primary_pos']

df1final = df1[vars1]

df1final.head()

,Name,ID,Age,Nationality,Overall,Potential,primary_pos
0,Cristiano Ronaldo,20801,32,Portugal,94,94,ST
1,L. Messi,158023,30,Argentina,93,93,RW
2,Neymar,190871,25,Brazil,92,94,LW
3,L. Suárez,176580,30,Uruguay,92,92,ST
4,M. Neuer,167495,31,Germany,92,92,GK


In [8]:
df1final.to_csv('FIFA_table1.csv')